This script is about showing you how to setup postgres access using the crobat package
The advantages to postgres is that its a long term data storage engine
While I need to work on archiving old data, rdms is great for doing panalysis 

In [12]:
# lets try set up the environment.
import os, sys
os.chdir('../..')
#os.chdir('..') # moves us 2 folders back into to allow for imports that users would use
wd = os.getcwd()
print(wd)
sys.path.append(wd)
#print(os.getcwd())
for i in sys.path:
    print(i)

/
/home/ivan/Documents/github/crobat/Jupyter nbs
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles/vscode_datascience_helpers
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles
/home/ivan/.vscode/extensions/ms-toolsai.jupyter-2021.6.999662501/pythonFiles/lib/python
/usr/lib/python38.zip
/usr/lib/python3.8
/usr/lib/python3.8/lib-dynload

/home/ivan/.local/lib/python3.8/site-packages
/usr/local/lib/python3.8/dist-packages
/usr/lib/python3/dist-packages
/home/ivan/.local/lib/python3.8/site-packages/IPython/extensions
/home/ivan/.ipython
/home/ivan/Documents/github
/home/ivan/Documents/github/crobat/crobat
/home/ivan/Documents/github/crobat/postgres
/home/ivan/Documents/github/tests
/home/ivan
/


In [13]:
# lets try importing the postgres setup 
from crobat.postgres import recorder as rec 
from copra.websocket import Channel, Client

In [14]:
# Step 1 : defining settings class
class input_args(object):
    def __init__(self, currency_pair='ETH-USD', position_range=5, recording_duration=5):
        self.currency_pair = currency_pair
        self.position_range = position_range
        self.recording_duration = recording_duration

settings = input_args()
print(dir(settings))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'currency_pair', 'position_range', 'recording_duration']


In [15]:
# step 2 constructing the ws connection
import asyncio
import nest_asyncio
nest_asyncio.apply()
loop = asyncio.get_event_loop()
channel = Channel('level2', settings.currency_pair) 
#channel2 = Channel('ticker', settings.currency_pair)
ws = rec.Ticker(loop, channel, settings)
#ws.subscribe(channel2)
try:
    loop.run_forever()
except KeyboardInterrupt:
    loop.run_until_complete(ws.close())
    loop.close()

Connecting to the PostgreSQL database...


TypeError: psql_setup_operations() takes no arguments